# Sub-queries and Nested Selects


In [16]:
# !docker stop mysql-container
# !docker start mysql-container
# !docker ps 

In [4]:
import mysql.connector as sql 
import os 
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv('/workspaces/IBM-DS-Course/.env')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

In [11]:
host = 'localhost'
port = 3306
db = 'HR'

In [37]:
conn = sql.connect(
    host=host,
    user=user,
    password=password,
    port=port
)
cursor = conn.cursor()

In [38]:
use = f'USE {db}'
cursor.execute(use)

Say you are asked to retrieve all employee records whose salary is lower than the average salary. You might use the following query to do this.

In [39]:
q1 = "SELECT * FROM EMPLOYEES WHERE SALARY < AVG(SALARY);"
cursor.execute(q1)
output = cursor.fetchall()
print(*[row for row in output], sep='\n')


DatabaseError: 1111 (HY000): Invalid use of group function

However, this query will generate an error stating, "Illegal use of group function." 

Here, the group function is AVG and **cannot be used directly** in the condition **since it has not been retrieved from the data**. 

Therefore, the condition will use a **sub-query** to retrieve the average salary information to compare the existing salary.

In [40]:
q1 = "SELECT * FROM EMPLOYEES WHERE SALARY < (SELECT AVG(SALARY) FROM EMPLOYEES);"
cursor.execute(q1)


In [41]:
output = cursor.fetchall()
print(*[row for row in output], sep='\n')

('E1003', 'Steve', 'Wells', 123458, '1980-10-08', 'M', '291 Springs, Gary,IL', 300, 50000, 30002, 5)
('E1004', 'Santosh', 'Kumar', 123459, '1985-07-20', 'M', '511 Aurora Av, Aurora,IL', 400, 60000, 30002, 5)
('E1005', 'Ahmed', 'Hussain', 123410, '1981-04-01', 'M', '216 Oak Tree, Geneva,IL', 500, 70000, 30001, 2)
('E1007', 'Mary', 'Thomas', 123412, '1975-05-05', 'F', '100 Rose Pl, Gary,IL', 650, 65000, 30003, 7)
('E1008', 'Bharath', 'Gupta', 123413, '1985-06-05', 'M', '145 Berry Ln, Naperville,IL', 660, 65000, 30003, 7)
('E1009', 'Andrea', 'Jones', 123414, '1990-09-07', 'F', '120 Fall Creek, Gary,IL', 234, 70000, 30003, 7)
('E1010', 'Ann', 'Jacob', 123415, '1982-03-30', 'F', '111 Britany Springs,Elgin,IL', 220, 70000, 30002, 5)


Now, consider executing a query that retrieves all employee records with EMP_ID, SALARY, and maximum salary as MAX_SALARY in every row. For this, the maximum salary must be queried and used as one of the columns. This can be done using the query below.

In [42]:
q2 = "SELECT EMP_ID, SALARY, (SELECT MAX(SALARY) FROM EMPLOYEES) AS MAX_SALARY FROM EMPLOYEES;"
cursor.execute(q2)


In [43]:
output = cursor.fetchall()
print(*[row for row in output], sep='\n')

('E1001', 100000, 100000)
('E1002', 80000, 100000)
('E1003', 50000, 100000)
('E1004', 60000, 100000)
('E1005', 70000, 100000)
('E1006', 90000, 100000)
('E1007', 65000, 100000)
('E1008', 65000, 100000)
('E1009', 70000, 100000)
('E1010', 70000, 100000)


Now, consider that you wish to extract the first and last names of the oldest employee. Since the oldest employee will be the one with the smallest date of birth, the query can be written as:

In [44]:
q3 = "SELECT F_NAME, L_NAME FROM EMPLOYEES WHERE B_DATE = (SELECT MIN(B_DATE) FROM EMPLOYEES);"
cursor.execute(q3)

In [45]:
output = cursor.fetchall()
print(*[row for row in output], sep='\n')

('Alice', 'James')


You may also use sub-queries to create derived tables, which can then be used to query specific information. Say you want to know the average salary of the top 5 earners in the company. You will first have to extract a table of the top five salaries as a table. From that table, you can query the average value of the salary. The query can be written as follows.

In [46]:
q4 = "SELECT AVG(SALARY) FROM (SELECT SALARY FROM EMPLOYEES ORDER BY SALARY DESC LIMIT 5) AS SALARY_TABLE;"
cursor.execute(q4)

In [47]:
output = cursor.fetchall()
print(*[row for row in output], sep='\n')

(Decimal('82000.0000'),)


Note that it is necessary to give an alias to any derived tables.

Write a query to find the average salary of the five least-earning employees.


In [64]:
p1 = "SELECT AVG(SALARY) FROM (SELECT SALARY FROM EMPLOYEES ORDER BY SALARY DESC LIMIT 5) AS TOP_SALARY;"
cursor.execute(p1)

In [65]:
output1 = cursor.fetchall()
print(*[row for row in output1], sep='\n')

(Decimal('82000.0000'),)


Write a query to find the records of employees older than the average age of all employees.

In [66]:
p2 = '''
SELECT * FROM EMPLOYEES 
WHERE DATEDIFF(CURRENT_DATE, B_DATE) > (
    SELECT AVG(DATEDIFF(CURRENT_DATE, B_DATE)) FROM EMPLOYEES 
    );
    '''
cursor.execute(p2)

In [67]:
output2 = cursor.fetchall()
print(*[row for row in output2], sep='\n')

('E1001', 'John', 'Thomas', 123456, '1976-09-01', 'M', '5631 Rice, OakPark,IL', 100, 100000, 30001, 2)
('E1002', 'Alice', 'James', 123457, '1972-07-31', 'F', '980 Berry ln, Elgin,IL', 200, 80000, 30002, 5)
('E1003', 'Steve', 'Wells', 123458, '1980-10-08', 'M', '291 Springs, Gary,IL', 300, 50000, 30002, 5)
('E1006', 'Nancy', 'Allen', 123411, '1978-06-02', 'F', '111 Green Pl, Elgin,IL', 600, 90000, 30001, 2)
('E1007', 'Mary', 'Thomas', 123412, '1975-05-05', 'F', '100 Rose Pl, Gary,IL', 650, 65000, 30003, 7)


From the Job_History table, display the list of Employee IDs, years of service, and average years of service for all entries.

In [70]:
p3 = '''
SELECT 
    EMPL_ID, 
    YEAR
    (  
        FROM_DAYS
                (
                DATEDIFF(CURRENT_DATE, START_DATE)
                )
    ), 
    (
        SELECT 
            AVG
            (
                YEAR
                (  
                    FROM_DAYS
                            (
                            DATEDIFF(CURRENT_DATE, START_DATE)
                            )
                )
            ) 
        FROM JOB_HISTORY
    )
    FROM JOB_HISTORY
    ;
    '''
    

cursor.execute(p3)

In [71]:
output3 = cursor.fetchall()
print(*[row for row in output3], sep='\n')

('E1001', 23, Decimal('18.6000'))
('E1002', 22, Decimal('18.6000'))
('E1003', 22, Decimal('18.6000'))
('E1004', 23, Decimal('18.6000'))
('E1005', 24, Decimal('18.6000'))
('E1006', 22, Decimal('18.6000'))
('E1007', 22, Decimal('18.6000'))
('E1008', 14, Decimal('18.6000'))
('E1009', 7, Decimal('18.6000'))
('E1010', 7, Decimal('18.6000'))


In [72]:
conn.commit()
conn.close()
